In [2]:
import pandas as pd
import unidecode 

In [3]:
def get_point(n):
    n = n.strip()
    n = n.replace('(', '')
    n = n.replace('\"', '')
    n = n.replace(')', '')
    n = n.replace(',', ' ')
    n = n.replace('_', ' ')
    n = n.replace('  ',' ')
    n = unidecode.unidecode(n)

    return n

In [10]:
df2 = pd.read_csv('../data/processed/final_2hop_v1.csv')
df3 = pd.read_csv('../data/processed/combined_3hop_v1.csv')

In [11]:
df2['source_clean'] = df2.source_text.apply(lambda x: get_point(x))
df2['cui_2_clean'] = df2.cui_2_text.apply(lambda x: get_point(x))
df2['cui_3_clean'] = df2.cui_3_text.apply(lambda x: get_point(x))

df2['rel_1_clean'] = df2['relation 1'].apply(lambda x: get_point(x))
df2['rel_2_clean'] = df2['relation 2'].apply(lambda x: get_point(x))
df2['rel_combine'] = df2.apply(lambda x: ' '.join([x['rel_1_clean'], x['rel_2_clean']]), axis = 1)

In [12]:
df2['source'] = df2.apply(lambda x: f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}']]", axis = 1)
df2['target'] = df2.apply(lambda x: f"{x['result']}", axis = 1)

In [13]:
df3['source_clean'] = df3.source_text.apply(lambda x: get_point(x))
df3['cui_2_clean'] = df3.cui_2_text.apply(lambda x: get_point(x))
df3['cui_3_clean'] = df3.cui_3_text.apply(lambda x: get_point(x))
df3['cui_4_clean'] = df3.cui_4_text.apply(lambda x: get_point(x))

df3['rel_1_clean'] = df3['relation'].apply(lambda x: get_point(x))
df3['rel_2_clean'] = df3['relation_2'].apply(lambda x: get_point(x))
df3['rel_3_clean'] = df3['relation_3'].apply(lambda x: get_point(x))
df3['rel_combine'] = df3.apply(lambda x: ' '.join([x['rel_1_clean'], 
                                                   x['rel_2_clean'],
                                                   x['rel_3_clean']]), axis = 1)

In [14]:
df3['source'] = df3.apply(lambda x: f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}'], ['{x['cui_3_clean']}', '{x['rel_3_clean']}', '{x['cui_4_clean']}']]", axis = 1)
df3['target'] = df3.apply(lambda x: f"{x['result']}", axis = 1)

In [16]:
# see which can be stratified
split_dict = dict(df2.rel_combine.value_counts() > 1)
df2['strat_split'] = df2.rel_combine.apply(lambda x: split_dict[x])

# separate dfs
df_str = df2[df2.strat_split == True].reset_index(drop = True)
df_nostr = df2[df2.strat_split == False].reset_index(drop = True)

from sklearn.model_selection import train_test_split
# Specify the feature (topic) column and the target variable (text) column.
feature_column = 'rel_combine'
text_column = 'result'

# Adjust the test_size and random_state as needed.
X_train, X_temp, y_train, y_temp = train_test_split(
    df_str[text_column],
    df_str[feature_column],
    test_size=0.2,  # Adjust the test size as needed.
    stratify=df_str[feature_column], # stratify by column
    random_state=42  # Adjust the random seed as needed.
)

# Split the remaining data into validation and testing sets (50% each).
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42  # Use the same random seed for consistency.
)

df_train1 = df_str[df_str.index.isin(X_train.index)]
df_val1 = df_str[df_str.index.isin(X_val.index)]
df_test1 = df_str[df_str.index.isin(X_test.index)]

# Adjust the test_size and random_state as needed.
X_train, X_temp, y_train, y_temp = train_test_split(
    df_nostr[text_column],
    df_nostr[feature_column],
    test_size=0.2,  # Adjust the test size as needed.
    random_state=42  # Adjust the random seed as needed.
)

# Split the remaining data into validation and testing sets (50% each).
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42  # Use the same random seed for consistency.
)

df_train2 = df_nostr[df_nostr.index.isin(X_train.index)]
df_val2 = df_nostr[df_nostr.index.isin(X_val.index)]
df_test2 = df_nostr[df_nostr.index.isin(X_test.index)]

df2_train = pd.concat([df_train1,df_train2])
df2_test = pd.concat([df_test1,df_test2])
df2_val = pd.concat([df_val1,df_val2])

In [18]:
# see which can be stratified
split_dict = dict(df3.rel_combine.value_counts() > 1)
df3['strat_split'] = df3.rel_combine.apply(lambda x: split_dict[x])

# separate dfs
df_str = df3[df3.strat_split == True].reset_index(drop = True)
df_nostr = df3[df3.strat_split == False].reset_index(drop = True)

from sklearn.model_selection import train_test_split
# Specify the feature (topic) column and the target variable (text) column.
feature_column = 'rel_combine'
text_column = 'result'

# Adjust the test_size and random_state as needed.
X_train, X_temp, y_train, y_temp = train_test_split(
    df_str[text_column],
    df_str[feature_column],
    test_size=0.2,  # Adjust the test size as needed.
    stratify=df_str[feature_column], # stratify by column
    random_state=42  # Adjust the random seed as needed.
)

# Split the remaining data into validation and testing sets (50% each).
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42  # Use the same random seed for consistency.
)

df_train1 = df_str[df_str.index.isin(X_train.index)]
df_val1 = df_str[df_str.index.isin(X_val.index)]
df_test1 = df_str[df_str.index.isin(X_test.index)]

# Adjust the test_size and random_state as needed.
X_train, X_temp, y_train, y_temp = train_test_split(
    df_nostr[text_column],
    df_nostr[feature_column],
    test_size=0.2,  # Adjust the test size as needed.
    random_state=42  # Adjust the random seed as needed.
)

# Split the remaining data into validation and testing sets (50% each).
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42  # Use the same random seed for consistency.
)

df_train2 = df_nostr[df_nostr.index.isin(X_train.index)]
df_val2 = df_nostr[df_nostr.index.isin(X_val.index)]
df_test2 = df_nostr[df_nostr.index.isin(X_test.index)]

df3_train = pd.concat([df_train1,df_train2])
df3_test = pd.concat([df_test1,df_test2])
df3_val = pd.concat([df_val1,df_val2])

In [19]:
df_train = pd.concat([df2_train, df3_train])
df_val = pd.concat([df2_val, df3_val])
df_test = pd.concat([df2_test, df3_test])

In [22]:
import json
import os

train_json = df_train.reset_index()[['source','target']]
val_json = df_val.reset_index()[['source','target']]
test_json = df_test.reset_index()[['source','target']]

data_dir = '../data/modelling/'

train_file = os.path.join(data_dir,'train_g2t_v1.json')
val_file = os.path.join(data_dir,'val_g2t_v1.json')
test_file = os.path.join(data_dir,'test_g2t_v1.json')


with open(train_file, 'w', encoding = 'utf-8') as train_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in train_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), train_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        train_json_file.write('\n')

with open(val_file, 'w', encoding = 'utf-8') as val_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in val_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), val_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        val_json_file.write('\n')

with open(test_file, 'w', encoding = 'utf-8') as test_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in test_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), test_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        test_json_file.write('\n')

In [ ]:
df_train.to_csv('../data/modelling/train_g2t_v1.csv', index = False)
df_test.to_csv('../data/modelling/val_g2t_v1.csv', index = False)
df_val.to_csv('../data/modelling/test_g2t_v1.csv', index = False)